## Setup

In [2]:
# Imports

import hfst_dev as hfst
import graphviz
import random

import itertools
import random
from nltk.probability import FreqDist
from nltk import CFG
from nltk import grammar, parse
from nltk.parse.generate import generate
from nltk.parse.util import load_parser

In [3]:
# Stream English 

istream = hfst.HfstInputStream('English')
assert istream.is_good() == True
English = istream.read()
istream.close()

In [4]:
# Set up definitions from phoneclass.fst 

defs = {'English' : English}

VowAA = hfst.regex('AA0 | AA1 | AA2', definitions=defs)
defs['VowAA'] = VowAA
VowAE = hfst.regex('AE0 | AE1 | AE2', definitions=defs)
defs['VowAE'] = VowAE
VowAH = hfst.regex('AH0 | AH1 | AH2', definitions=defs)
defs['VowAH'] = VowAH
VowAO = hfst.regex('AO0 | AO1 | AO2', definitions=defs)
defs['VowAO'] = VowAO
VowAW = hfst.regex('AW0 | AW1 | AW2', definitions=defs)
defs['VowAW'] = VowAW
VowAY = hfst.regex('AY0 | AY1 | AY2', definitions=defs)
defs['VowAY'] = VowAY
VowEH = hfst.regex('EH0 | EH1 | EH2', definitions=defs)
defs['VowEH'] = VowEH
VowER = hfst.regex('ER0 | ER1 | ER2', definitions=defs)
defs['VowER'] = VowER
VowEY = hfst.regex('EY0 | EY1 | EY2', definitions=defs)
defs['VowEY'] = VowEY
VowIH = hfst.regex('IH0 | IH1 | IH2', definitions=defs)
defs['VowIH'] = VowIH
VowIY = hfst.regex('IY0 | IY1 | IY2', definitions=defs)
defs['VowIY'] = VowIY
VowOW = hfst.regex('OW0 | OW1 | OW2', definitions=defs)
defs['VowOW'] = VowOW
VowOY = hfst.regex('OY0 | OY1 | OY2', definitions=defs)
defs['VowOY'] = VowOY
VowUH = hfst.regex('UH0 | UH1 | UH2', definitions=defs)
defs['VowUH'] = VowUH
VowUW = hfst.regex('UW0 | UW1 | UW2', definitions=defs)
defs['VowUW'] = VowUW

Vow0 = hfst.regex('AH0| IH0| ER0| IY0| OW0| AA0| EH0| UW0| AE0| AO0| AY0| EY0| AW0| UH0| OY0', definitions=defs)
defs['Vow0'] = Vow0
Vow1 = hfst.regex('EH1| AE1| AA1| IH1| IY1| EY1| OW1| AO1| AY1| AH1| UW1| ER1| AW1| UH1| OY1', definitions=defs)
defs['Vow1'] = Vow1
Vow2 = hfst.regex('EH2| EY2| AE2| AY2| AA2| IH2| OW2| IY2| AO2| UW2| AH2| AW2| ER2| UH2| OY2', definitions=defs)
defs['Vow2'] = Vow2

Vow = hfst.regex('Vow0 | Vow1 | Vow2', definitions=defs)
defs['Vow'] = Vow

Nas = hfst.regex('N | M | NG', definitions=defs)
defs['Nas'] = Nas

Phone = hfst.regex('AH0| N| S| L| T| R| K| D| IH0| M| Z| ER0| IY0| B| EH1| P| AE1| AA1| IH1| F| G| V| IY1| NG| HH| EY1| W| SH| OW1| OW0| AO1| AY1| AH1| UW1| JH| Y| CH| AA0| ER1| EH2| EY2| AE2| AY2| AA2| EH0| IH2| TH| AW1| OW2| UW0| IY2| AO2| AE0| UH1| AO0| AY0| UW2| AH2| EY0| OY1| AW2| DH| ZH| ER2| UH2| AW0| UH0| OY2| OY0', definitions = defs)
defs['Phone'] = Phone

Cons = hfst.regex('[Phone - Vow]', definitions = defs)
defs['Cons'] = Cons

## Creating Classes of Words Based on Stress

Only created classes we could use: for example s0s0 wouldnt be helpful to us

### One Syllable Words

#### Stressed

In [5]:
expr = '[English .o. [[ Cons* Vow1 Cons* ].l]].u'
n = hfst.regex(expr, definitions=defs)
defs["s1"] = n

#### Unstressed

In [6]:
expr = '[English .o. [[ Cons* Vow0 Cons* ].l]].u'
n = hfst.regex(expr, definitions=defs)
defs["s0"] = n

### Two Syllable Words

#### Main stress first

In [7]:
expr = '[English .o. [[Cons* Vow1 Cons* Vow0 Cons*]].l].u'
n = hfst.regex(expr, definitions=defs)
defs["s1s0"] = n

expr = '[English .o. [[Cons* Vow1 Cons* Vow2 Cons*]].l].u'
m = hfst.regex(expr, definitions=defs)
defs["s1s2"] = m

#### Main stress second 

In [8]:
expr = '[English .o. [[Cons* Vow0 Cons* Vow1 Cons*]].l].u'
n = hfst.regex(expr, definitions=defs)
defs["s0s1"] = n

expr = '[English .o. [[Cons* Vow2 Cons* Vow1 Cons*]].l].u'
m = hfst.regex(expr, definitions=defs)
defs["s2s1"] = m

### Three Syllable Words

#### Stressed, unstressed, stressed

In [9]:
expr = '[English .o. [[ Cons* Vow1 Cons* Vow0 Cons* Vow1 Cons*]].l].u'
m = hfst.regex(expr, definitions=defs)
defs["s1s0s1"] = m

expr = '[English .o. [[ Cons* Vow1 Cons* Vow0 Cons* Vow2 Cons*]].l].u'
n = hfst.regex(expr, definitions=defs)
defs["s1s0s2"] = n

expr = '[English .o. [[ Cons* Vow1 Cons* Vow2 Cons* Vow1 Cons*]].l].u'
o = hfst.regex(expr, definitions=defs)
defs["s1s2s1"] = o

expr = '[English .o. [[ Cons* Vow2 Cons* Vow0 Cons* Vow2 Cons*]].l].u'
p = hfst.regex(expr, definitions=defs)
defs["s2s0s2"] = p

expr = '[English .o. [[ Cons* Vow2 Cons* Vow0 Cons* Vow1 Cons*]].l].u'
q = hfst.regex(expr, definitions=defs)
defs["s2s0s1"] = q

expr = '[English .o. [[ Cons* Vow1 Cons* Vow2 Cons* Vow2 Cons*]].l].u'
r = hfst.regex(expr, definitions=defs)
defs["s1s2s2"] = r

expr = '[English .o. [[ Cons* Vow2 Cons* Vow2 Cons* Vow1 Cons*]].l].u'
s = hfst.regex(expr, definitions=defs)
defs["s2s2s1"] = s

#### Unstressed, stressed, unstressed

In [10]:
expr = '[English .o. [[ Cons* Vow0 Cons* Vow1 Cons* Vow0 Cons*]].l].u'
n = hfst.regex(expr, definitions=defs)
defs["s0s1s0"] = n

expr = '[English .o. [[ Cons* Vow0 Cons* Vow1 Cons* Vow2 Cons*]].l].u'
m = hfst.regex(expr, definitions=defs)
defs["s0s1s2"] = m

expr = '[English .o. [[ Cons* Vow0 Cons* Vow2 Cons* Vow0 Cons*]].l].u'
o = hfst.regex(expr, definitions=defs)
defs["s0s2s0"] = o

expr = '[English .o. [[ Cons* Vow2 Cons* Vow1 Cons* Vow0 Cons*]].l].u'
p = hfst.regex(expr, definitions=defs)
defs["s2s1s0"] = p

expr = '[English .o. [[ Cons* Vow2 Cons* Vow1 Cons* Vow2 Cons*]].l].u'
q = hfst.regex(expr, definitions=defs)
defs["s2s1s2"] = q

## Generate Iambic Pentameter

In [11]:
# Sample input and output

def sample_input(x,n=8,cycles=3):
        x2 = x.copy()
        x2.input_project()
        x2.minimize()
        word = None
        dictionary = ['upright', 'abundant', 'annoying', 'adversely', 'quickly', 'very', 'and', 'or', 'but', 'every', 'some', 'no', 'a', 'that', 'who', 'whom', 'risotto', 'aqueduct', 'aqueducts', 'wok', 'woks', 'guitar', 'guitars', 'trustee', 'trustees', 'accredits', 'accredit']
        while word == None or word not in dictionary:
            word = random.sample(set(x2.extract_paths(max_cycles=3).keys()),n)[0].replace('|', '')
        return word
def sample_output(x,n=8,cycles=3):
        x2 = x.copy()
        x2.output_project()
        x2.minimize()
        return(random.sample(set(x2.extract_paths(max_cycles=3).keys()),n))

In [12]:
def sample(wordClasses : list, defs) -> (str, str):
    # [wordClasses] a LIST of [word class, frequency] lists, with word classes defined in [defs]
    # Frequencies should add up to 1
    # 
    # Returns: (word class, sample)
    r = random.random()
    for wordClass in wordClasses:
        r -= wordClass[1]
        if r < 0:
            return (wordClass[0], sample_input(hfst.regex(wordClass[0], definitions=defs), n=1))

In [13]:
def classesToList(lst, wordClasses : dict):
    result = []
    sumFreq = 0
    for wc in lst:
        result.append([wc, wordClasses[wc]])
        sumFreq += wordClasses[wc]
    for i in range(len(result)):
        result[i][1] /= sumFreq
    return result

In [14]:
def generate_iambs(wordClasses : dict, defs):
    # [wordClasses] a DICTIONARY mapping word classes (defined in [defs]) to frequencies
    # Frequencies should add up to 1
    # Each element: s1 (primary), s2 (secondary), s0 (unstressed)
    syllables = []
    words_out = []
    index = 0
    while index < 10:
        if index == 0:
            preLst = ["s0", "s0s1", "s2s1", "s0s1s0", "s0s1s2", "s0s2s0", "s2s1s0", "s2s1s2"]
        elif index == 8:
            if index % 2 == 0:
                preLst = ["s0", "s0s1"]
                if syllables[index - 1] == "s1":
                    preLst.extend(["s2s1"])
        elif index == 9:
            preLst = ["s1"]
        else:
            # Unstressed
            if index % 2 == 0:
                preLst = ["s0", "s0s1", "s0s1s0", "s0s1s2", "s0s2s0"]
                if syllables[index - 1] == "s1":
                    preLst.extend(["s2s1", "s2s1s0", "s2s1s2"])
            # Stressed
            else:
                preLst = ["s1", "s1s0", "s1s2", "s1s0s1", "s1s0s2", "s1s2s1", "s1s2s2"]
                if syllables[index - 1] == "s0":
                    preLst.extend(["s2s0s2", "s2s0s1", "s2s2s1"])
                    
        lst = classesToList(preLst, wordClasses)      
        wordClass, word = sample(lst, defs)
        wordSyl = wordClass.split("s")
        for syl in wordSyl:
            if syl != "":   
                syllables.append("s" + syl) 
                index += 1
        words_out.append(word)
        
    return words_out

In [15]:
wordClasses = {"s0": 1/16, "s1": 1/16, "s0s1": 1/16, "s1s0": 1/16, "s2s1": 1/16, "s1s2": 1/16, "s0s1s0": 1/16, "s0s1s2": 1/16, "s0s2s0": 1/16, "s2s1s0": 1/16, "s2s1s2": 1/16, "s1s0s1": 1/16, "s1s0s2": 1/16, "s1s2s1": 1/16, "s2s0s2": 1/16, "s2s0s1": 1/16}

In [16]:
def check_grammatical(s):
    try:
        t = next(p.parse(s))
        return True
    except StopIteration:
        return False
    except ValueError:
        return False

In [17]:
p = load_parser("grammar.fcfg", trace=0, cache=False)
g = p.grammar()

In [18]:
print(English)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
sent = None
while sent == None or not check_grammatical(sent):
    sent = generate_iambs(wordClasses, defs)
    print(sent)

KeyError: 's1s2s2'

In [20]:
s = 'some actor whom a black cat that chases a shiny rat quickly chases chases small dogs'.split()
t = next(p.parse(s))
t.draw()

ValueError: Grammar does not cover some of the input words: "'actor', 'black', 'cat', 'chases', 'shiny', 'rat', 'chases', 'chases', 'small', 'dogs'".

In [21]:
from random import randint

class wordIterable:
    def __init__(self, file, n):
        p1 = load_parser(file, trace=0, cache=False)
        g1 = p1.grammar()
        self.parser = p1
        self.grammar = g1
        self.n = n
        self.sents = []
        gen1 = generate(g1, depth=2*n)
        s_temp = list(gen1)
        for s in s_temp:
            if len(s) == self.n and self.check_grammatical(s) and s not in self.sents:
                self.sents.append(s)
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if len(self.sents) > 0:
            i = randint(0, len(self.sents) - 1)
            sent = self.sents[i]
            del self.sents[i]
            return sent
        else:
            raise StopIteration
    
    def check_grammatical(self, s):
        try:
            t = next(self.parser.parse(s))
            return True
        except StopIteration:
            return False

In [22]:
gen = generate(g, depth=4)

generated_sents = []
for i in range(50):
    s = next(gen)
    while not check_grammatical(s) or s in generated_sents:
        s = next(gen)
    generated_sents.append(s)
    print(s)

['every', 'risotto', 'is', 'upright']
['every', 'risotto', 'is', 'abundant']
['every', 'risotto', 'is', 'annoying']
['every', 'risotto', 'was', 'upright']
['every', 'risotto', 'was', 'abundant']
['every', 'risotto', 'was', 'annoying']
['every', 'aqueduct', 'is', 'upright']
['every', 'aqueduct', 'is', 'abundant']
['every', 'aqueduct', 'is', 'annoying']
['every', 'aqueduct', 'was', 'upright']
['every', 'aqueduct', 'was', 'abundant']
['every', 'aqueduct', 'was', 'annoying']
['every', 'wok', 'is', 'upright']
['every', 'wok', 'is', 'abundant']
['every', 'wok', 'is', 'annoying']
['every', 'wok', 'was', 'upright']
['every', 'wok', 'was', 'abundant']
['every', 'wok', 'was', 'annoying']
['every', 'guitar', 'is', 'upright']
['every', 'guitar', 'is', 'abundant']
['every', 'guitar', 'is', 'annoying']
['every', 'guitar', 'was', 'upright']
['every', 'guitar', 'was', 'abundant']
['every', 'guitar', 'was', 'annoying']
['every', 'trustee', 'is', 'upright']
['every', 'trustee', 'is', 'abundant']
['every

In [23]:
# Apparently encoding lists as dictionary.keys() makes everything run faster
wordClasses = {"s0": 1/16, "s1": 1/16, "s0s1": 1/16, "s1s0": 1/16, "s2s1": 1/16, "s1s2": 1/16, "s0s1s0": 1/16, "s0s1s2": 1/16, "s0s2s0": 1/16, "s2s1s0": 1/16, "s2s1s2": 1/16, "s1s0s1": 1/16, "s1s0s2": 1/16, "s1s2s1": 1/16, "s2s0s2": 1/16, "s2s0s1": 1/16}
rhymePatterns = {"AA": 0, "AE": 0, "AH": 0, "AO": 0, "AW": 0, "AY": 0, "EH": 0, "ER": 0, "EY": 0, "IH": 0, "IY": 0, "OW": 0, "OY": 0, "UH": 0, "UW": 0}

In [24]:
tail = 0
with open('dummy.fcfg', 'r') as f:
    lines = []
    for line in f:
        if line != '\n':
            tail += 1
            lines.append(line.strip())

FileNotFoundError: [Errno 2] No such file or directory: 'dummy.fcfg'

In [ ]:
def check_sentence_left(left):
    if '[' not in left and left == 'S' or left.split('[')[0] == 'S':
        return True
    return False

In [ ]:
def rule_add_stress(left, right, sp):
# left: one symbol
# right: [symbol1, symbol2] or [symbol1]
# sp: stress patterns for symbols 1 and 2 [sp1, sp2]
# return (left, right)

    if check_sentence_left(left):
        left_stress = left
    else:
        if '[' in left:
            ind = left.find('[')
            left_stress = left[:ind] + '_' + sp[0] + sp[1] + left[ind:]
        else:
            left_stress = left + '_' + sp[0] + sp[1]
        
    right_stress = []
    for i in range(len(right)):
        if '[' in right[i]:
            ind = right[i].find('[')
            right_stress.append(right[i][:ind] + '_' + sp[i] + right[i][ind:])
        else:
            right_stress.append(right[i] + '_' + sp[i])
            
    return left_stress + ' -> ' + right_stress[0] + ' ' + (right_stress[1] if len(right) > 1 else '')
    #left_stress, right_stress

In [ ]:
def rule_add_rhyme(left, right, rp):
# left: one symbol
# right: [symbol1, symbol2] or [symbol1]
# rp: rhyme patterns for symbols 1 and 2 [rp1, rp2]

    if check_sentence_left(left):
        left_rhyme = left
    else:
        if '[' in left:
            ind = left.find('[')
            left_rhyme = left[:ind] + '_' + rp[-1] + left[ind:]
        else:
            left_rhyme = left + '_' + rp[-1]
    
    right_rhyme = []
    for i in range(len(right)):
        if '[' in right[i]:
            ind = right[i].find('[')
            right_rhyme.append(right[i][:ind] + '_' + rp[i] + right[i][ind:])
        else:
            right_rhyme.append(right[i] + '_' + rp[i])
    
    return left_rhyme + ' -> ' + right_rhyme[0] + ' ' + (right_rhyme[1] if len(right) > 1 else '')

In [ ]:
def check_meter_add(sp1, sp2):
    sp1end = 's' + sp1.split('s')[-1]
    sp2begin = 's' + sp2.split('s')[1]
    if sp1end == 's0':
        if sp2begin == 's1':
            return True
        if sp2begin == 's0':
            return False
    if sp1end == 's1':
        if sp2begin == 's0':
            return True
        if sp2begin == 's1':
            return False
    if sp1end == 's2':
        sp1pu = 's' + sp1.split('s')[-2]
        if sp1pu == 's0':
            if sp2begin == 's0':
                return True
            else:
                return False
        elif sp1pu == 's1':
            if sp2begin == 's1':
                return True
            else:
                return False
        else:
            raise Exception("Two s2's in a row in sp1")
    if sp2begin == 's2':
        sp2sec = 's' + sp2.split('s')[2]
        if sp2sec == 's0':
            if sp1end == 's0':
                return True
            else:
                return False
        elif sp2sec == 's1':
            if sp1end == 's1':
                return True
            else:
                return False
        else:
            raise Exception("Two s2's in a row in sp2")

In [ ]:
def check_iambic_pent(left, sp1, sp2):
    if check_sentence_left(left):
        if not check_meter_add(sp1, sp2):
            return False
        if len(sp1 + sp2) != 20:
            return False
        if 's' + sp1.split('s')[1] == 's1':
            return False
        if 's' + sp1.split('s')[1] == 's2' and len(sp1.split('s')[1:]) > 1 and 's' + sp1.split('s')[2] == 's0':
            return False
        return True
    else:
        return check_meter_add(sp1, sp2)

In [ ]:
def concat(wordClasses):
    result = []
    for sp in wordClasses:
        result.append(sp)
    
    for sp1 in wordClasses:
        for sp2 in wordClasses:
            if check_meter_add(sp1, sp2):
                result.append(sp1 + sp2)
    
    limitHit = False
    for sp in result:
        if len(sp) >= 10:
            limitHit = True
            break
            
    if not limitHit:
        result = concat(result)

    return result

In [ ]:
ls = wordClasses.keys()
wc = concat(ls)

In [ ]:
def rewrite_grammar(wordClasses):
    grammar_section = False
    lexical_section = False
    index = 0;

    while grammar_section == False:
        if lines[index] == '# Grammar Rules':
            grammar_section = True
        index += 1

    new_grammar = set()
    while lexical_section == False:
        if lines[index] == '# Lexical Rules':
            lexical_section = True
        if ' -> ' in lines[index]:
            left = lines[index].split(' -> ')[0]
            right = lines[index].split(' -> ')[1].split(' ')
            for sp1 in wordClasses:
                for sp2 in wordClasses:
                    if check_iambic_pent(left, sp1, sp2):
                        #for rp1 in rhymePatterns:
                            #for rp2 in rhymePatterns:
                                #sleft, sright = rule_add_stress(left, right, [sp1, sp2])
                                #rule_add_rhyme(sleft, sright, [rp1, rp2])
                        new_grammar.add(rule_add_stress(left, right, [sp1, sp2]))
        index += 1

    lexical_rules = []
    while index < tail:
        lexical_rules.append(lines[index])
        index += 1

    
    with open('dummynew.fcfg', 'w') as f:
        f.write("% start S\n")
        for rule in new_grammar:
            f.write(rule)
            f.write('\n')
        for rule in lexical_rules:
            f.write(rule)
            f.write('\n')

In [ ]:
rewrite_grammar(wc)

In [ ]:
len(new_grammar)

In [ ]:
pd = load_parser("dummynew.fcfg", trace=0, cache=False)
gd = pd.grammar()

In [ ]:
s = ['the', 'purple', 'dog', 'inflates', 'the', 'purple', 'dog']
td = next(pd.parse(s))
td = next(pd.parse(s))
#td = next(pd.parse(s))
#td = next(pd.parse(s))
td.draw()

In [26]:
def check_grammatical(p, s):
    try:
        t = next(p.parse(s))
        return True
    except StopIteration:
        return False
    except ValueError:
        return False

gen = generate(gd, depth=20)

s = next(gen)
generated_sents = []
for i in range(5):
    while not check_grammatical(pd, s) or s in generated_sents:
        s = next(gen)
    print(s)
    generated_sents.append(s)

NameError: name 'gd' is not defined

In [27]:
generated_sents = []
generated_set = set(generated_sents )

In [33]:
rhyme_dict = {
    "snowfall" : 'AA', 
    "sakura" : 'AA', 
    "enlarge": 'AA',
    "jumpstart" : 'AA',
    "unharmed" : 'AA',
    "remark" : 'AA', 
    "wasp" : 'AA',
    "resolved" :'AA',
    "jock" : 'AA',
    "charm" : 'AA',
    "bask" : 'AE',
    "replant" : 'AE',
    "chasse" : 'AE',
    "hunchback" : 'AE',
    "woodland" : 'AE',
    "thrash" : 'AE',
    "catch" : 'AE',
    "ads" : 'AE',
    "fad" : 'AE',
    "mailbag" : 'AE',
    "oration" : 'AH',
    "straightened" : 'AH',
    "walnut" : 'AH',
    "abhorrent" : 'AH',
    "credence" : 'AH',
    "megaton" : 'AH',
    "puma" : 'AH',
    "stuffs" : 'AH',
    "junction" : 'AH',
    "patients" : 'AH',
    "troughs" : 'AO',
    "frauds" : 'AO',
    "meatballs" : 'AO',
    "imports" : 'AO',
    "lords" : 'AO',
    "hoar" : 'AO',
    "malformed" : 'AO',
    "billboard" : 'AO',
    "shorn" : 'AO',
    "thorns" : 'AO',
    "bloodhound" : 'AW',
    "crowns" : 'AW',
    "blackout" : 'AW',
    "reroute" : 'AW', 
    "loud" : 'AW',
    "hometown" : 'AW',
    "scowl" : 'AW',
    "countdown" : 'AW',
    "rouse" : 'AW',
    "mount" : 'AW',
    "bloodhound" : ''
    , "crowns", "blackout", "reroute", "loud", "hometown", "scowl", "countdown", "rouse", "mount"
}
    

    
AW = ["bloodhound", "crowns", "blackout", "reroute", "loud", "hometown", "scowl", "countdown", "rouse", "mount"]
AY = ["devise", "privatize", "bribe", "modernize", "coincide", "chimes", "deprived", "reunite", "apprise", "knifelike"]
EH = ["doorsteps", "aspects", "flare", "sleepwear", "pens", "pastel", "bullpen", "pipette"]
ER = ["rewired", "spindler", "harvesters", "thunders", "lowered", "gander", "prisoners", "trimmer", "scholar", "modern"]
EY = ["prepaid", "gateways", "blockades", "replace", "cliched", "acclimate", "drain", "birthdays", "upscale", "sedate"]
IH = ["hallways", "parades", "dislocate", "hurricane", "escape", "downplay", "shortchange", "lace", "days"]
IY = ["coyote", "squeaky", "delete", "cheek", "cream", "blackberry", "publicly", "blatantly"]
OW = ["yolks", "chrome", "intone", "pronto", "sorrow", "disowned", "potatoes", "mole", "notes"]
OY = ["decoy", "convoy", "noise", "annoy", "purloin", "steroid", "datapoint", "boy", "tabloids", "soy"]
UH = ["wolves", "underwood", "scrapbooks", "cooked", "schedules", "cookbooks", "endure", "understood", "rook", "woods"]
UW = ["balloons", "typhoons", "duped", "croon", "loon", "resume", "ingenue", "remove", "lawsuit", "troops"]
}

TypeError: unhashable type: 'list'

In [37]:
# generate sonnet
import random 
r = random.randint(0, 34)
r

4

In [25]:
# generate sonnet
import random 



def generate_abab_stanza(generated_sents, rhyme_dict, poem_so_far=[]):
    
    r = random.randint(0, len(generated_sents))

    sents_list = poem_so_far
    sents_list.append([generated_sents[r]])
    a = rhyme_dict[generated_sents[r][-1]]

    r = random.randint(0, len(generated_sents))

    sents_list.append(generated_sents[r])
    b = rhyme_dict[generated_sents[r][-1]]

    i = 0
    while i < len(generated_sents):
        if rhyme_dict[generated_sents[i][-1]] == a and generated_sents[i] not in sents_list:
            sents_list.append(generated_sents[i])
            break;
        i += 1
        if i == len(generated_sents):
            print("Sorry! Could not find a rhyme.")

    j = 0
    while i < len(generated_sents):
        if rhyme_dict[generated_sents[j][-1]] == b and generated_sents[j] not in sents_list:
            sents_list.append(generated_sents[j])
            break; 
        j += 1
        if j == len(generated_sents):
            print("Sorry! Could not find a rhyme.")
    
    return sents_list 

def generate_gg_couplet(generated_sents, rhyme_dict, poem_so_far):
    
    r = random.randint(0, len(generated_sents))
    sents_list = poem_so_far
    sents_list.append([generated_sents[r]])
    g = rhyme_dict[generated_sents[r][-1]]
    
    i = 0
    while i < len(generated_sents):
        if rhyme_dict[generated_sents[i][-1]] == g and generated_sents[i] not in sents_list:
            sents_list.append(generated_sents[i])
            break;
        i +=1
        if i == len(generated_sents):
            print("Sorry! Could not find a rhyme.")
        
    return sents_list

SyntaxError: unexpected EOF while parsing (1309789209.py, line 4)